### memo

- モジュールの数を複数にしていきたい
- モジュールの配置を可視化するプログラムは欲しい

## 参考文献
1. King, D. et al, 2004, “Sandia Photovoltaic Array Performance Model”, SAND Report 3535, Sandia National Laboratories, Albuquerque, NM.
    - pvlib.temperature.sapm_cell() の定義式

In [6]:
import pvlib
import pandas as pd

In [18]:
# 場所の定義（例: 東京）
location = pvlib.location.Location(latitude=35.6895, longitude=139.6917, tz='Asia/Tokyo')

# 時間範囲の定義
times = pd.date_range(start='2024-08-01', end='2024-08-02', freq='60min', tz=location.tz)

# 太陽位置の計算
solar_position = location.get_solarposition(times)

# 傾斜と方位の設定 (例: 傾斜角30度、南向き)
surface_tilt = 30  # 傾斜角 (度)
surface_azimuth = 180  # 方位角 (度), 南向きは180度

# 標準大気条件下での直達放射と散乱放射を計算
dni_extra = pvlib.irradiance.get_extra_radiation(times)
aoi = pvlib.irradiance.aoi(surface_tilt, surface_azimuth, solar_position['zenith'], solar_position['azimuth'])

total_irradiance = pvlib.irradiance.get_total_irradiance(
    surface_tilt,
    surface_azimuth,
    solar_position['zenith'],
    solar_position['azimuth'],
    dni=1000,  # 直達放射照度 (W/m^2)
    ghi=800,  # 全放射照度 (W/m^2)
    dhi=100,  # 散乱放射照度 (W/m^2)
)

# 太陽光モジュールとインバータのパラメータ
module_parameters = {
    'Name': 'Example Module',
    'BIPV': 'N',
    'Date': '2021',
    'T_NOCT': 45.0,
    'Bvmpo': -0.0034,     # 最大電力点電圧の温度係数 (V/°C)
    'Bvoco': -0.0036,     # 開放電圧の温度係数 (V/°C)
    'Bvmp': -0.0034,      # 電流依存の最大電力点電圧係数 (V/°C)
    'Mbvmp': 0.0,         # 電流依存の最大電力点電圧の修正係数
    'Mbvoc': 0.0,         # 電流依存の開放電圧の修正係数
    'Voco': -0.002,          # 開放電圧の温度係数 (V/°C)
    'IXO': 0.0005,          # 短絡電流の温度係数 (A/°C)
    'IXXO': 0.0001,         # 短絡電流の温度応答の補正係数 (A/°C)
    'FD': 0.02,             # セル温度の短絡電流への影響 (A/°C)
    'N': 1.0,   
    'A_c': 1.94,
    'Cells_in_Series': 60,
    'Isco': 9.43,
    'Impo': 8.87,
    'Vmpo': 31.1,
    'alpha_sc': 0.004539,
    'beta_oc': -0.123,
    'a_ref': 1.473,
    'I_L_ref': 9.38,
    'I_o_ref': 1.92e-10,
    'R_s': 0.43,
    'R_sh_ref': 1100,
    'Adjust': 8.7,
    'gamma_r': -0.0045,
    'Aisc': 0.004,
    'Aimp': -0.0004,
    'A0': 0.035,            # 係数 A0 (特定のモデルによる)
    'A1': -0.0002,          # 係数 A1
    'A2': 0.000005,         # 係数 A2
    'A3': -0.0000001,       # 係数 A3
    'A4': 0.000000001,      # 係数 A4
    'B0': 0.03,             # 係数 B0
    'B1': -0.0001,          # 係数 B1
    'B2': 0.000002,         # 係数 B2
    'B3': -0.00000001,      # 係数 B3
    'B4': 0.0000000001,     # 係数 B4
    'B5': -0.000000000001,  # 係数 B5
    'C0': 0.1,              # 係数 C0
    'C1': -0.0005,          # 係数 C1
    'C2': 0.00001,          # 係数 C2
    'C3': -0.0000002,       # 係数 C3
    'C4': 0.000000002,      # 係数 C4
    'C5': -0.00000000002,   # 係数 C5
    'C6': 0.0000000000001,  # 係数 C6
    'C7': -0.000000000000001 # 係数 C7
}

inverter_parameters = {
    'Name': 'Example Inverter',
    'Vac': 230,
    'Vdcmax': 500,
    'Idcmax': 15,
    'Mppt_low': 200,
    'Mppt_high': 480,

    'Paco': 5000,  # インバータのAC出力定格 [W]
    'Pdco': 5200,  # リファレンス電圧 Vdco で Paco 出力を達成するためのDC入力電力 [W]
    'Vdco': 400,   # Paco 出力を達成する際のDC電圧 [V]
    'Pso': 20,     # インバータの起動に必要なDC電力、または低電力レベルでのインバータ効率に強く影響する自己消費電力 [W]
    'C0': 0.0005,  # リファレンス運転条件でのDC電力とAC電力の関係の曲率（放物線）の定義パラメータ [1/W]
    'C1': 0.001,   # DC電圧入力によりPdcoを線形に変化させる経験係数 [1/V]
    'C2': 0.002,   # DC電圧入力によりPsoを線形に変化させる経験係数 [1/V]
    'C3': 0.0002,  # DC電圧入力によりC0を線形に変化させる経験係数 [1/V]
    'Pnt': 0.1     # インバータの夜間（ナイトタレ）に消費されるAC電力 [W]
}

# PVシステムの定義
system = pvlib.pvsystem.PVSystem(
    surface_tilt=surface_tilt,
    surface_azimuth=surface_azimuth,
    module_parameters=module_parameters,
    inverter_parameters=inverter_parameters
)

# セル温度の計算
cell_temperature = pvlib.temperature.sapm_cell(
    poa_global=total_irradiance['poa_global'],
    temp_air=25,  # 周囲温度 (°C)
    wind_speed=1,  # 風速 (m/s)
    a=-3.56,
    b=-0.075,
    deltaT=3
)

# DC出力の計算 (例: Sandia PV Array Performance Model)
dc_output = system.sapm(
    effective_irradiance=total_irradiance['poa_direct'],  # 有効入射照度
    temp_cell=cell_temperature
)

# AC出力の計算
ac_output = pvlib.inverter.sandia(dc_output['v_mp'], dc_output['p_mp'], inverter_parameters)

# 結果を表示
print(f"Total AC energy produced: {ac_output.sum()} W")

Total AC energy produced: -0.22827506437397901 W
